Data Ingest
======

The data is a key component of any machine learning algorithm.
The ingest of the data,
including the cleanup and labelling stages,
is often the most time consuming.
It requires taking the raw data,
in our case Molecular Dyanmics trajectories, and
computing or extracting the quantities which will be used
for the Machine Learning.

In this example we will be using the relative orientations
of neighbouring molecules for our classification.
So by the end of this workbook we will have
a collection of data containing;

- the relative orientations of up to 6 nearest neighbours,
- the known classification, and
- temperature 

The input files
--------------

The input data for this tutorial is in the folder `data`,
which contains a series of `.gsd` files.
This is the default file format of [Hoomd-blue](http://glotzerlab.engin.umich.edu/hoomd-blue/)
a molecular dynamics package designed to run on GPUs
from the Glotzer group at the University of Michigan.

To read these files we are going to use the [gsd](http://gsd.readthedocs.io/en/latest/) package
that the glotzer group provide.
To read files from other simulation packages [MDAnalysis](https://www.mdanalysis.org/)
is a python package that will read nearly any file type.
To read all the files that we want,
we are going to use the [pathlib](https://docs.python.org/3/library/pathlib.html) module 
from python's (>= 3.4) standard library.

In [1]:
from pathlib import Path

import gsd.hoomd

In [2]:
# Create a Path object pointing to the directory holding
# all the input files we will be using
directory = Path('../data/simulations/melting')
# Search for all files in the above directory which end in .gsd
input_files = directory.glob('*.gsd')

# input files is a generator, essentially a list of files,
# so we can iterate through each individually
for fname in input_files:
    # This is a context manager. We open the file and assign
    # it to the variable traj.
    with gsd.hoomd.open(str(fname)) as trj:
        # This extracts the first (and only) frame of the trajectory
        snap = trj[0]

For more information on context managers in Python,
Jeff Knupp has a [good explanation](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/).

### Collate Simulation Data

This code is enough to read all the input files,
so the next step is to actually do something useful
with each of the files.

Within each of the filenames I have included 
the parameters of the simulation.

### Labelling Data

This has given us sufficient information about the simulation,
we now need to work out the classification of each molecule.
The configurations that I have prepared consist of two phases,
the middle 2/3 in the $x$ and $y$ directions is crystalline
while the remainer of the simulation cell is liquid.
This is probably easiest to understand using a picture;

In [3]:
from sdanalysis.figures.configuration import plot_frame
from sdanalysis.util import get_filename_vars
from bokeh.plotting import show, output_notebook
output_notebook()

show(plot_frame(snap))

Loading BokehJS ...

AttributeError: 'Snapshot' object has no attribute 'orientation'

We can define molecules that have an $x$ position in the range
$
-L_x/3 < x < L_x/3
$
and have a $y$ position in the range
$
-L_y/3 < y < L_y/3
$
as being crystalline, with the crystal structure taken from the filename. 
The remaining molecules can be classed as the generic liquid.
As before we can write a simple function
that takes a snapshot and the cyrstal structure,
returning the annotated classification of all molecules in the simulation.
At the interface of the two phases
the classification is not well defined,
so for the purposes of training and testing, 
I am going to remove molecules within a distance of 3.5,
a bit over 1 neighbour shell,
to the boundary.

In [ ]:
import numpy as np

def classify_mols(snapshot, crystal, boundary_buffer=3.5):
    """Classify molecules as crystalline, amorphous or boundary."""
    # This is the method of extracting the positions from a gsd snapshot
    position = snapshot.particles.position
    # This gets the details of the box from the simulation
    box = snapshot.configuration.box
    
    # All axes have to be True, True == 1, use product for logical and operation
    is_crystal = np.product(np.abs(position) < box[:3]/3, axis=1).astype(bool)
    boundary = np.logical_and(np.product(np.abs(position) < box[:3]/3+boundary_buffer, axis=1),
                              np.product(np.abs(position) > box[:3]/3-boundary_buffer, axis=1))
    
    # Create classification array
    classification = np.full(snapshot.particles.N, 'liq', dtype='<U4')
    classification[is_crystal] = crystal
    classification[boundary] = None
    return classification

### Compute training features

The final step is to compute the nearest neighbours for each of the molecules.
Most of the work of this function is done
using a function I wrote, `relative_orientations`.
This function uses the the kdtree algorithm from scipy to compute the neighbours efficiently.
A side effect of using this function is that the neighbours are returned in order of increasing distance.
Then computes the relative orientation of the neighbour orientations using quaternion maths.

It should be noted that this the relative orientations function requires the orientations
to be expressed as quaternions. 

The parameters to the relative_orientations function, `max_radius` and `max_neighbours`
are passed to the algorithm computing the neighbour lists.
`max_radius` defines the maximum distance to search for neighbours,
beyond this distance molecules are not considered neighbours.
The `max_neighbours` parameter defines the maximum number of neighbours to find.
Where there are more molecules within the `max_radius` 
only the closest `max_neighbours` are returned.

In [ ]:
from sdanalysis.order import relative_orientations

def compute_orientations(snapshot):
    """Compute the orientation of 6 nearest neighbours from a gsd snapshot."""
    # I am assuming an orthorhombic simulation cell
    box = snapshot.configuration.box[:3]
    return relative_orientations(box=box,
                                 position=snapshot.particles.position,
                                 orientation=snapshot.particles.orientation,
                                 max_radius=3.5,
                                 max_neighbours=6)

In [ ]:
# Check our function works 
compute_orientations(snap)

Loading the Data
----------------

Now with all the parts in place we can load all the data into a pandas DataFrame.
Taking the code we had at the start to read in all the snapshots,
we can now apply the functions we have just written,

- `get_sim_params`,
- `classify_mols`, and
- `compute_orientations`

to process the data into a succinct and usable form.

In [ ]:
import pandas

input_files = directory.glob('*.gsd')
all_dataframes = []

for fname in input_files:
    with gsd.hoomd.open(str(fname)) as trj:
        snap = trj[0]
        # Get simulation parameters
        params = get_filename_vars(fname)
        # Classify all molecules
        classes = classify_mols(snap, params.crystal)
        # Compute the orientations
        orientations = compute_orientations(snap)
        
        # Create dataframe
        df = pandas.DataFrame({
            'temperature': params.temperature,
            'pressure': params.pressure,
            'crystal': params.crystal,
            'class': classes,
            'orient0': orientations[:, 0],
            'orient1': orientations[:, 1],
            'orient2': orientations[:, 2],
            'orient3': orientations[:, 3],
            'orient4': orientations[:, 4],
            'orient5': orientations[:, 5],
        })
        
        # Remove molecules close to interface
        df = df[df['class'] != 'None']
        
        all_dataframes.append(df)

# Collate list of dataframes into single large dataframe
training_dataset = pandas.concat(all_dataframes)

# Save dataset to file
training_dataset.to_hdf('.../data/analysis/training_data.h5', key='trimer')

In [ ]:
# Check the dataframe contains reasonable data
training_dataset.head()

Finding a Machine Learning Model
=================

There are many different types of models we can use for classification,
each of these models have types of problems they are well suited to.
The goal of this notebook is to identify algorithms 
that will effectively classify our dataset
which we can then investigate further.

### Collating the models

The first step here is creating a list of models we would like to test.
An excellent property of scikit-learn is 
that all the algorithms have the same API,
allowing us to treat them all in the same way. 

This is not an exhastive list of all the possible classifiers in scikit-learn,
just a smattering for comparison.
For a more exhastive list check out [the scikit-learn documentation](http://scikit-learn.org/stable/supervised_learning.html#supervised-learning),
and feel free to add more to the list.

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

ml_models = {
    'LR': LogisticRegression(),
    'SGD': SGDClassifier(tol=1e-3, max_iter=1000),
    'LDA': LinearDiscriminantAnalysis(),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'NB': GaussianNB(),
    'SVM': SVC(),
    'NN': MLPClassifier()
}

### Loading the training data

We need to load in the training dataset we created in the first notebook.
At this point we are interested in two sets of data,

- $X$, the input data which is the orientation of the six nearest neighbours
- $Y$, the true labelled classification of the data.

In [ ]:
import pandas

training_dataset = pandas.read_hdf('../data/analysis/training_data.h5', key='trimer')
X = training_dataset[['orient0', 'orient1', 'orient2', 'orient3', 'orient4', 'orient5']].values
Y = training_dataset['class'].values
classes = training_dataset['class'].unique()

### Testing the Models

With a collection of models to test,
we now need some method of testing the models to compare them.
To perform the initial screening of datasets
we are going to break our training data into two groups,

- the training set, comprising 80% of the molecules
- the validation set, comprising the remaining 20%.

This division of the dataset gives us a set of data 
previously unseen by the algorithms,
giving us a method of testing whether
the algorithm is acutally learning the underlying features,
or just 'memorising' the training data.
This division of data will be through a random selection
so as not to bias the selection of data.

In [ ]:
from sklearn import model_selection

validation_size = 0.20
seed = 7

selected = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
X_train, X_validation, Y_train, Y_validation = selected

To get an idea of the models which
warrant a further investigation,
we can iterate through each of our models.
Each model is scored by breaking the training data into `n_splits`,
using one of these splits for testing and
the remaining splits for training.
This process is referred to as *cross validation*
and typically the number of splits is 10.
For the purposes of this running in a reasonable amount of time,
`n_splits` is set to 2.

In [ ]:
scoring='accuracy'
n_splits = 2
# Typically n_splits would be 10 but it runs much slower
#n_splits = 10

# Iterate through each model in our dictionary of models
for name, model in ml_models.items():
    kfold = model_selection.KFold(n_splits=n_splits, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    print(f'{name:5s}: {cv_results.mean():.5f} ± {cv_results.std():.5f}')

Out of all the algorithms tested, 
there are three that stand out 

- K-Nearest Neighbours (KNN),
- Support Vector Machine (SVM), and
- Neural Network (NN)

with accuracies in excess of 95%. 

So with these three algorithms it is likely worth
tweaking the algorithms slightly from 
the defualt paramters in an effort to improve performance.
It is also worth understanding which classes
each of these algorithms is strongest at classifying.
For this additional data we are going to be using a [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).
In a confusion matrix, 
the diagonal elements represent the correct classifications,
while the off diagonal elements are the values 
which were incorrectly classified.

Below we have a handy function from the scikit-learn documentation
that will nicely plot the confusion matrix as a heatmap. 

In [ ]:
import itertools

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

%matplotlib inline

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

It is this point where our breaking the data into
training and validation sets becomes useful.
We can train a classifier using the training set,
then generate predictions of the validation dataset
using the known values of the validation data
to generate the confusion matrix.

In [ ]:
knn = ml_models['KNN']
knn.fit(X_train, Y_train)
predictions = knn.predict(X_validation)
plot_confusion_matrix(confusion_matrix(Y_validation, predictions, labels=classes), classes, normalize=True)

In [ ]:
svm = ml_models['SVM']
svm.fit(X_train, Y_train)
predictions = svm.predict(X_validation)
plot_confusion_matrix(confusion_matrix(Y_validation, predictions, labels=classes), classes, normalize=True)

In [ ]:
nn = ml_models['NN']
nn.fit(X_train, Y_train)
predictions = nn.predict(X_validation)
plot_confusion_matrix(confusion_matrix(Y_validation, predictions, labels=classes), classes, normalize=True)

It is interesting to note that all of the models
have the most difficulty with the liquid/crystal characterisation,
with the largest proportion of false positives being 
crystal incorrectly classified as liquid. 
To make this model we have created persistent
it needs to be saved which is done using `joblib`.

In [ ]:
from sklearn.externals import joblib
joblib.dump(ml_models['KNN'], '../data/analysis/knn-model.pkl') 

ML in Production
=========

So far we have been looking at building a machine learning model,
while this is nice we actually want to be able to use it
to perform useful science.

This notebook will demonstrate the application of the machine learning algorithm
that we built in the first part of the tutorial to
the classification of a previously unseen configuration.

Loading the Model
----------------

At the end of the [previous notebook](02_Lets_Find_A_Model.ipynb)
we saved the model as a python pickle using scikit-learn's `joblib` library.
This converted the in memory object that represented the trained state
of the machine learning algorithm into a form that could be saved to disk.
By reading the file `knn-model.pkl` from disk,
we can load the trained K-Nearest Neighbours model.

In [ ]:
from sklearn.externals import joblib
model = joblib.load('data/knn-model.pkl')
model

With the model now loaded we need some data to apply it to.
For this we need to load a configuration and
compute the relative orientation of all the neareset neighbours.
With the nearest neighbour orientations comptued,
all that is left to do is use the model to predict the classes.

In [ ]:
from sdanalysis.order import relative_orientations

with gsd.hoomd.open('data/unknown/configuration.gsd') as trj:
    snap = trj[0]
orientations = relative_orientations(snap.configuration.box,
                           snap.particles.position,
                           snap.particles.orientation,
                           max_neighbours=6,
                           max_radius=3.5)
classes = model.predict(orientations)
classes

With the classification in hand we can perform any number of analyses.
Below I am taking the classes array and using pandas
to generate a histogram of the data.
We can see that most of the configuration is liquid,
there are large amounts of both the p2 and p2gg crystals, and 
a small number of pg crystals.

In [ ]:
import pandas
%matplotlib inline

pandas.Series(classes).value_counts().plot(kind='bar')

If we want a better understanding of the configuration,
we can plot the configuration in the notebook,
colouring by the classification.
For this I am using [bokeh](https://bokeh.pydata.org/en/latest/)
since it allows interacting with the dataset by panning a zooming,
allowing for both an overview and investigation in a single figure. 

In [ ]:
import numpy as np
from bokeh.plotting import show, output_notebook, figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Colorblind4 as palette
from sdanalysis.figures.configuration import plot_circles, snapshot2data

# Output the final figure to the jupyter notebook
output_notebook()

# The colours we will assign each class
class_colours = {
    'liq': palette[0],
    'p2': palette[1],
    'p2gg': palette[2],
    'pg': palette[3],
}

# Convert the class strings to a colour for plotting
coloured_classes = [class_colours[c] for c in classes]

# Convert the snapshot to format ready for plotting 
data = snapshot2data(snap) 
data['colour'] = np.tile(coloured_classes, 3)
data['label'] = np.tile(classes, 3)
source = ColumnDataSource(data)

# Create the figure
p = figure(aspect_scale=1, match_aspect=True, width=920, height=800, active_scroll='wheel_zoom')
p.circle('x', 'y', radius='radius', source=source, color='colour', legend='label')

show(p)